In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split, Subset
from torchvision import transforms, models
from torchvision.models import EfficientNet_V2_S_Weights
from PIL import Image
import pandas as pd
import numpy as np
import os
import csv
from torch.optim.lr_scheduler import ReduceLROnPlateau
from tqdm.notebook import tqdm
import platform
import multiprocessing

class StabilityDataset(Dataset):
    def __init__(self, csv_file, img_dir, target_feature, transform=None, augment=False, image_size=224, zoom_proportion=0.15):
        self.stability_data = pd.read_csv(csv_file)
        self.img_dir = img_dir
        self.target_feature = target_feature
        self.transform = transform
        self.augment = augment
        self.image_size = image_size
        self.zoom_proportion = zoom_proportion
        self.augmented_indices = self._create_augmented_indices() if augment else None
        
        # Automatically determine class mapping
        self.class_mapping = self._create_class_mapping()
    
    def _create_class_mapping(self):
        unique_values = sorted(self.stability_data[self.target_feature].unique())
        return {value: index for index, value in enumerate(unique_values)}


    def _create_augmented_indices(self):
        base_indices = list(range(len(self.stability_data)))
        flipped_indices = [idx + len(self.stability_data) for idx in base_indices]
        zoomed_indices = [idx + 2 * len(self.stability_data) for idx in base_indices]
        zoomed_flipped_indices = [idx + 3 * len(self.stability_data) for idx in base_indices]
        return base_indices + flipped_indices + zoomed_indices + zoomed_flipped_indices

    def __len__(self):
        return len(self.stability_data) * 4 if self.augment else len(self.stability_data)

    def __getitem__(self, idx):
        if self.augment:
            original_idx = idx % len(self.stability_data)
            augmentation = idx // len(self.stability_data)
        else:
            original_idx = idx
            augmentation = 0

        img_name = str(self.stability_data.iloc[original_idx, 0])
        img_path = os.path.join(self.img_dir, img_name)
        if not os.path.exists(img_path):
            img_path = os.path.join(self.img_dir, f"{img_name}.jpg")
        
        image = Image.open(img_path).convert('RGB')
        
        target_value = self.stability_data.iloc[original_idx][self.target_feature]
        target_class = self.class_mapping[target_value]

        if self.augment:
            if augmentation in [1, 3]:
                image = image.transpose(Image.Transpose.FLIP_LEFT_RIGHT)
            if augmentation in [2, 3]:
                width, height = image.size
                crop_size = int(min(width, height) * (1 - self.zoom_proportion))
                left = (width - crop_size) // 2
                top = (height - crop_size) // 2
                right = left + crop_size
                bottom = top + crop_size
                image = image.crop((left, top, right, bottom))

        image = image.resize((self.image_size, self.image_size), Image.BILINEAR)
        
        if self.transform:
            image = self.transform(image)

        return image, torch.tensor(target_class, dtype=torch.long)

class StabilityPredictor(nn.Module):
    def __init__(self, num_classes, dropout_rate=0.3):
        super(StabilityPredictor, self).__init__()
        weights = EfficientNet_V2_S_Weights.DEFAULT
        self.efficientnet = models.efficientnet_v2_s(weights=weights)
        num_ftrs = self.efficientnet.classifier[1].in_features
        self.efficientnet.classifier = nn.Sequential(
            nn.Dropout(p=dropout_rate, inplace=True),
            nn.Linear(num_ftrs, num_classes)
        )

    def forward(self, x):
        return self.efficientnet(x)
    
# Function to train the model
def train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, num_epochs, patience, device):
    model.to(device)
    
    best_val_loss = float('inf')
    epochs_no_improve = 0
    best_model = None
    
    for epoch in range(num_epochs):
        print(f'Epoch {epoch+1}/{num_epochs}')
        
        # Training phase
        model.train()
        train_loss, train_acc = run_epoch(model, train_loader, criterion, optimizer, device, is_training=True)
        
        # Validation phase
        model.eval()
        val_loss, val_acc = run_epoch(model, val_loader, criterion, optimizer, device, is_training=False)
        
        # Learning rate scheduler step
        scheduler.step(val_loss)

        print(f'Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}%')
        print(f'Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.2f}%')
        print(f'Learning Rate: {optimizer.param_groups[0]["lr"]:.6f}')
        print('-' * 60)

        # Early stopping check
        if val_loss <= best_val_loss:
            best_val_loss = val_loss
            epochs_no_improve = 0
            best_model = model.state_dict()
        else:
            epochs_no_improve += 1

        if epochs_no_improve == patience or val_acc > 99.99:
            print(f'Early stopping triggered after {epoch + 1} epochs')
            model.load_state_dict(best_model)
            break

    return model

# Function to run a single epoch (training or validation)
def run_epoch(model, data_loader, criterion, optimizer, device, is_training=True):
    running_loss = 0.0
    correct = 0
    total = 0

    # Create progress bar
    progress_bar = tqdm(data_loader, desc="Training" if is_training else "Validating")

    for inputs, labels in progress_bar:
        inputs, labels = inputs.to(device), labels.to(device)
        
        if is_training:
            optimizer.zero_grad()
        
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        
        if is_training:
            loss.backward()
            optimizer.step()
        
        running_loss += loss.item() * inputs.size(0)
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

        # Update progress bar
        progress_bar.set_postfix({
            'loss': f'{loss.item():.4f}',
            'acc': f'{100. * correct / total:.2f}%'
        })
    
    epoch_loss = running_loss / len(data_loader.dataset)
    epoch_acc = 100. * correct / total

    return epoch_loss, epoch_acc

# Function to calculate dataset statistics
def calculate_stats(dataset):
    loader = DataLoader(dataset, batch_size=100, num_workers=0, shuffle=False)
    mean = 0.
    std = 0.
    for images, _ in loader:
        batch_samples = images.size(0)
        images = images.view(batch_samples, images.size(1), -1)
        mean += images.mean(2).sum(0)
        std += images.std(2).sum(0)
    
    mean /= len(dataset)
    std /= len(dataset)
    return mean, std

# Function to get optimal number of workers for data loading
def get_optimal_num_workers():
    # Windows can't do multiprocessing
    if platform.system() == 'Windows':
        return 0
    else:
        return multiprocessing.cpu_count()
    
def main(config):
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

    # Read the CSV file once
    train_data = pd.read_csv(config['train_csv'])

    # Automatically determine features and their number of classes
    features = {}
    for column in train_data.columns:
        if column == 'cam_angle': #not in ['id', 'stable_height', 'shapeset']:  # Exclude non-feature columns
            unique_values = train_data[column].nunique()
            features[column] = unique_values

    for feature, num_classes in features.items():
        print(f"\nTraining model for feature: {feature}")
        
        # Update config for the current feature
        config['target_feature'] = feature
        config['num_classes'] = num_classes
        config['model_save_path'] = f'stability_predictor_{feature}.pth'
        config['predictions_save_path'] = f'predictions_{feature}.csv'

        # Create datasets
        full_dataset = StabilityDataset(csv_file=config['train_csv'], 
                                        img_dir=config['train_img_dir'], 
                                        target_feature=config['target_feature'],
                                        transform=transforms.ToTensor(),
                                        augment=False,
                                        image_size=config['image_size'])
        
        # Split dataset
        dataset_size = len(full_dataset)
        indices = list(range(dataset_size))
        np.random.shuffle(indices)
        split = int(np.floor(config['val_ratio'] * dataset_size))
        train_indices, val_indices = indices[split:], indices[:split]

        train_subset = Subset(full_dataset, train_indices)

        print("Calculating training dataset statistics...")
        train_mean, train_std = calculate_stats(train_subset)

        data_transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize(mean=train_mean, std=train_std),
        ])

        train_dataset = StabilityDataset(csv_file=config['train_csv'], 
                                         img_dir=config['train_img_dir'], 
                                         target_feature=config['target_feature'],
                                         transform=data_transform,
                                         augment=config['use_augmentation'],
                                         image_size=config['image_size'],
                                         zoom_proportion=config['zoom_proportion'])
        train_dataset = Subset(train_dataset, [i for i in range(len(train_dataset)) if i % len(full_dataset) in train_indices])

        val_dataset = StabilityDataset(csv_file=config['train_csv'], 
                                       img_dir=config['train_img_dir'], 
                                       target_feature=config['target_feature'],
                                       transform=data_transform,
                                       augment=False,
                                       image_size=config['image_size'])
        val_dataset = Subset(val_dataset, val_indices)

        # Create data loaders
        train_loader = DataLoader(train_dataset, batch_size=config['batch_size'], shuffle=True, num_workers=config['num_workers'])
        val_loader = DataLoader(val_dataset, batch_size=config['batch_size'], shuffle=False, num_workers=config['num_workers'])

        # Initialize model, criterion, optimizer, and scheduler
        model = StabilityPredictor(num_classes=config['num_classes'], dropout_rate=config['dropout_rate'])
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.Adam(model.parameters(), lr=config['learning_rate'])
        scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=config['lr_factor'], patience=config['lr_patience'])

        # Train model
        print('Training...')
        model = train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, 
                            num_epochs=config['num_epochs'], patience=config['early_stopping_patience'], device=device)

        torch.save(model.state_dict(), config['model_save_path'])
        print(f"Model for {feature} saved to {config['model_save_path']}")

# Configuration
config = {
    'train_csv': './COMP90086_2024_Project_train/train.csv',
    'train_img_dir': './COMP90086_2024_Project_train/train',
    'test_csv': './COMP90086_2024_Project_test/test.csv',
    'test_img_dir': './COMP90086_2024_Project_test/test',
    'image_size': 224,
    'val_ratio': 0.1,
    'use_augmentation': False,
    'zoom_proportion': 0.15,
    'batch_size': 16,
    'num_workers': get_optimal_num_workers(),
    'dropout_rate': 0.3,
    'learning_rate': 0.001,
    'lr_factor': 0.1,
    'lr_patience': 2,
    'num_epochs': 30,
    'early_stopping_patience': 3,
}

main(config)

Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x7b42351feb30>>
Traceback (most recent call last):
  File "/workspace/COMP90086_Project/.venv/lib/python3.10/site-packages/ipykernel/ipkernel.py", line 775, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(
KeyboardInterrupt: 


In [10]:
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import pandas as pd
import os
from tqdm import tqdm
from PIL import Image
import logging
import traceback

logging.basicConfig(level=logging.INFO)

class TestStabilityDataset(Dataset):
    def __init__(self, csv_file, img_dir, transform=None, image_size=224):
        self.data = pd.read_csv(csv_file)
        self.img_dir = img_dir
        self.transform = transform
        self.image_size = image_size
        logging.info(f"TestStabilityDataset initialized with {len(self.data)} samples")

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        try:
            img_name = str(self.data.iloc[idx]['id'])
            img_path = os.path.join(self.img_dir, f"{img_name}.jpg")
            
            if not os.path.exists(img_path):
                logging.error(f"Image file not found: {img_path}")
                return None, img_name

            image = Image.open(img_path).convert('RGB')
            image = image.resize((self.image_size, self.image_size), Image.BILINEAR)
            
            if self.transform:
                image = self.transform(image)

            return image, img_name
        except Exception as e:
            logging.error(f"Error processing image at index {idx}: {e}")
            logging.error(traceback.format_exc())
            return None, str(self.data.iloc[idx]['id'])

def collate_fn(batch):
    batch = list(filter(lambda x: x[0] is not None, batch))
    if len(batch) == 0:
        return torch.Tensor(), []
    return torch.stack([item[0] for item in batch]), [item[1] for item in batch]

def load_model(model_path, num_classes):
    try:
        model = StabilityPredictor(num_classes)
        model.load_state_dict(torch.load(model_path, map_location=torch.device('cpu'), weights_only=True))
        model.eval()
        return model
    except Exception as e:
        logging.error(f"Error loading model from {model_path}: {e}")
        logging.error(traceback.format_exc())
        return None

def predict(model, data_loader, device):
    predictions = []
    img_names = []
    try:
        with torch.no_grad():
            for inputs, names in tqdm(data_loader, desc="Predicting"):
                if inputs.numel() == 0:
                    continue
                inputs = inputs.to(device)
                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                predictions.extend(preds.cpu().numpy())
                img_names.extend(names)
    except Exception as e:
        logging.error(f"Error during prediction: {e}")
        logging.error(traceback.format_exc())
    return predictions, img_names

def main():
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    logging.info(f"Using device: {device}")
    
    # Configuration
    test_csv = './COMP90086_2024_Project_test/test.csv'
    test_img_dir = './COMP90086_2024_Project_test/test'
    train_csv = './COMP90086_2024_Project_train/train.csv'
    batch_size = 32
    num_workers = 0  # Set to 0 to debug DataLoader issues
    image_size = 224

    # Load test and train data
    test_data = pd.read_csv(test_csv)
    train_data = pd.read_csv(train_csv)
    logging.info(f"Loaded {len(test_data)} test samples and {len(train_data)} train samples")

    # Prepare data transform
    data_transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])

    # Dictionary to store predictions
    all_predictions = {}

    # Get all features except 'id' and 'stable_height' from train_data
    features = [col for col in train_data.columns if col not in ['id', 'stable_height']]
    logging.info(f"Features to predict: {features}")

    # Create test dataset and dataloader
    test_dataset = TestStabilityDataset(csv_file=test_csv, 
                                        img_dir=test_img_dir, 
                                        transform=data_transform,
                                        image_size=image_size)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, 
                             num_workers=num_workers, collate_fn=collate_fn)

    # Iterate through each feature
    for feature in features:
        logging.info(f"Predicting {feature}...")
        
        # Load the corresponding model
        model_path = f'stability_predictor_{feature}.pth'
        if not os.path.exists(model_path):
            logging.warning(f"Model for {feature} not found. Skipping.")
            continue

        # Load model
        num_classes = len(train_data[feature].unique())
        logging.info(f"Number of classes for {feature}: {num_classes}")
        model = load_model(model_path, num_classes)
        if model is None:
            logging.warning(f"Failed to load model for {feature}. Skipping.")
            continue

        model.to(device)

        # Make predictions
        logging.info(f"Starting predictions for {feature}")
        predictions, img_names = predict(model, test_loader, device)
        logging.info(f"Completed predictions for {feature}. Got {len(predictions)} predictions.")

        if len(predictions) == 0:
            logging.warning(f"No predictions were made for {feature}. Skipping.")
            continue

        # Convert numeric predictions back to original classes
        class_mapping = {i: class_name for i, class_name in enumerate(sorted(train_data[feature].unique()))}
        predictions = [class_mapping[pred] for pred in predictions]

        # Store predictions
        all_predictions[feature] = dict(zip(img_names, predictions))
        logging.info(f"Stored predictions for {feature}")

    # Create a new dataframe with 'id' from the original test data
    result_df = pd.DataFrame({'id': test_data['id']})

    # Add predictions to the result dataframe
    for feature in features:
        if feature in all_predictions:
            result_df[feature] = result_df['id'].astype(str).map(all_predictions[feature])
            logging.info(f"Added predictions for {feature} to result DataFrame")
        else:
            logging.warning(f"No predictions found for {feature}")

    # Save the imputed test data
    output_path = 'test_imputed.csv'
    result_df.to_csv(output_path, index=False)
    logging.info(f"Predictions saved to {output_path}")

    # Log a sample of the predictions
    logging.info(f"Sample of predictions:\n{result_df.head()}")

    # Log value counts for each feature
    for feature in features:
        logging.info(f"Value counts for {feature}:\n{result_df[feature].value_counts()}")

if __name__ == "__main__":
    main()

INFO:root:Using device: cuda:0
INFO:root:Loaded 1920 test samples and 7680 train samples
INFO:root:Features to predict: ['shapeset', 'type', 'total_height', 'instability_type', 'cam_angle']
INFO:root:TestStabilityDataset initialized with 1920 samples
INFO:root:Predicting shapeset...
INFO:root:Number of classes for shapeset: 2
INFO:root:Starting predictions for shapeset
Predicting: 100%|██████████| 60/60 [00:07<00:00,  7.87it/s]
INFO:root:Completed predictions for shapeset. Got 1920 predictions.
INFO:root:Stored predictions for shapeset
INFO:root:Predicting type...
INFO:root:Number of classes for type: 2
INFO:root:Starting predictions for type
Predicting: 100%|██████████| 60/60 [00:07<00:00,  8.03it/s]
INFO:root:Completed predictions for type. Got 1920 predictions.
INFO:root:Stored predictions for type
INFO:root:Predicting total_height...
INFO:root:Number of classes for total_height: 5
INFO:root:Starting predictions for total_height
Predicting: 100%|██████████| 60/60 [00:08<00:00,  7.03